In [1]:
def write_answer(answer, number):
    print 'write answer #{0}: {1}'.format(number, answer)
    with open("week_4_part_1_answer_{0}.txt".format(number), "w") as fout:
        fout.write(str(answer))

Загрузите датасет digits с помощью функции load_digits из sklearn.datasets и подготовьте матрицу признаков X и ответы на обучающей выборке y (вам потребуются поля data и target в объекте, который возвращает load_digits).

In [2]:
from sklearn import datasets, cross_validation, tree
from sklearn.model_selection import cross_val_score
import numpy as np

/Users/andrejezov/anaconda3/envs/py2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
digits_dataset = datasets.load_digits()

X_data = digits_dataset.data
y = digits_dataset.target

print X_data[0:2]
print y[0:2]

[[ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0. 13. 15. 10. 15.  5.  0.  0.  3.
  15.  2.  0. 11.  8.  0.  0.  4. 12.  0.  0.  8.  8.  0.  0.  5.  8.  0.
   0.  9.  8.  0.  0.  4. 11.  0.  1. 12.  7.  0.  0.  2. 14.  5. 10. 12.
   0.  0.  0.  0.  6. 13. 10.  0.  0.  0.]
 [ 0.  0.  0. 12. 13.  5.  0.  0.  0.  0.  0. 11. 16.  9.  0.  0.  0.  0.
   3. 15. 16.  6.  0.  0.  0.  7. 15. 16. 16.  2.  0.  0.  0.  0.  1. 16.
  16.  3.  0.  0.  0.  0.  1. 16. 16.  6.  0.  0.  0.  0.  1. 16. 16.  6.
   0.  0.  0.  0.  0. 11. 16. 10.  0.  0.]]
[0 1]


Для оценки качества далее нужно будет использовать cross_val_score из sklearn.cross_validation с параметром cv=10. Эта функция реализует k-fold cross validation c k равным значению параметра cv. Мы предлагаем использовать k=10, чтобы полученные оценки качества имели небольшой разброс, и было проще проверить полученные ответы. На практике же часто хватает и k=5. Функция cross_val_score будет возвращать numpy.ndarray, в котором будет k чисел - качество в каждом из k экспериментов k-fold cross validation. Для получения среднего значения (которое и будет оценкой качества работы) вызовите метод .mean() у массива, который возвращает cross_val_score.

In [4]:
k = 10
cv = 10

С небольшой вероятностью вы можете натолкнуться на случай, когда полученное вами качество в каком-то из пунктов не попадет в диапазон, заданный для правильных ответов - в этом случае попробуйте перезапустить ячейку с cross_val_score несколько раз и выбрать наиболее «типичное» значение. Если это не помогает, то где-то была допущена ошибка.

Если вам захочется ускорить вычисление cross_val_score - можете попробовать использовать параметр n_jobs, но будьте осторожны: в одной из старых версий sklearn была ошибка, которая приводила к неверному результату работы cross_val_score при задании n_jobs отличным от 1. Сейчас такой проблемы возникнуть не должно, но проверить, что все в порядке, не будет лишним.

# 1.

Создайте DecisionTreeClassifier с настройками по умолчанию и измерьте качество его работы с помощью cross_val_score. Эта величина и будет ответом в пункте 1.

In [5]:
clf = tree.DecisionTreeClassifier()
result = cross_val_score(clf, X_data, y, cv=cv)

In [6]:
write_answer(np.mean(result), 1)

write answer #1: 0.832093263708


Качество работы одного решающего дерева обычно получается не очень высоким, но на этом датасете получилось неплохо. Обратите внимание - дерево вполне могло получиться сильно переобученным, ведь мы не ограничивали его глубину.

# 2.

Воспользуйтесь BaggingClassifier из sklearn.ensemble, чтобы обучить бэггинг над DecisionTreeClassifier. Используйте в BaggingClassifier параметры по умолчанию, задав только количество деревьев равным 100.

Качество классификации новой модели - ответ в пункте 2. Обратите внимание, как соотносится качество работы композиции решающих деревьев с качеством работы одного решающего дерева.

In [7]:
from sklearn.ensemble import BaggingClassifier

clf = tree.DecisionTreeClassifier()
bagging_classifier = BaggingClassifier(clf, n_estimators = 100)
result = cross_val_score(bagging_classifier, X_data, y, cv=cv)

In [8]:
write_answer(np.mean(result), 2)

write answer #2: 0.927677521505


# 3.

Теперь изучите параметры BaggingClassifier и выберите их такими, чтобы каждый базовый алгоритм обучался не на всех d признаках, а на 
d
d
​	  случайных признаков. Качество работы получившегося классификатора - ответ в пункте 3. Корень из числа признаков - часто используемая эвристика в задачах классификации, в задачах регрессии же часто берут число признаков, деленное на три. Но в общем случае ничто не мешает вам выбирать любое другое число случайных признаков.

In [17]:
max_features = int(np.sqrt(X_data.shape[1]))
print max_features

8


In [18]:
clf = tree.DecisionTreeClassifier()
bagging_classifier = BaggingClassifier(clf, max_features=max_features, n_estimators = 100)
result = cross_val_score(bagging_classifier, X_data, y, cv=cv)

In [19]:
write_answer(np.mean(result), 3)

write answer #3: 0.928192999446


# 4.

Наконец, давайте попробуем выбирать случайные признаки не один раз на все дерево, а при построении каждой вершины дерева. Сделать это несложно: нужно убрать выбор случайного подмножества признаков в BaggingClassifier и добавить его в DecisionTreeClassifier. Какой параметр за это отвечает, можно понять из документации sklearn, либо просто попробовать угадать (скорее всего, у вас сразу получится). Попробуйте выбирать опять же 
d
d
​	  признаков. Качество полученного классификатора на контрольной выборке и будет ответом в пункте 4.

In [32]:
clf = tree.DecisionTreeClassifier(max_features=max_features)
bagging_classifier = BaggingClassifier(clf, n_estimators = 100, bootstrap=False)
result = cross_val_score(bagging_classifier, X_data, y, cv=cv)

In [33]:
write_answer(np.mean(result), 4)

write answer #4: 0.956175064271


# 5.

Полученный в пункте 4 классификатор - бэггинг на рандомизированных деревьях (в которых при построении каждой вершины выбирается случайное подмножество признаков и разбиение ищется только по ним). Это в точности соответствует алгоритму Random Forest, поэтому почему бы не сравнить качество работы классификатора с RandomForestClassifier из sklearn.ensemble. Сделайте это, а затем изучите, как качество классификации на данном датасете зависит от количества деревьев, количества признаков, выбираемых при построении каждой вершины дерева, а также ограничений на глубину дерева. Для наглядности лучше построить графики зависимости качества от значений параметров, но для сдачи задания это делать не обязательно.

На основе наблюдений выпишите через пробел номера правильных утверждений из приведенных ниже в порядке возрастания номера (это будет ответ в п.5)

1) Случайный лес сильно переобучается с ростом количества деревьев

2) При очень маленьком числе деревьев (5, 10, 15), случайный лес работает хуже, чем при большем числе деревьев

3) С ростом количества деревьев в случайном лесе, в какой-то момент деревьев становится достаточно для высокого качества классификации, а затем качество существенно не меняется.

4) При большом количестве признаков (для данного датасета - 40, 50) качество классификации становится хуже, чем при малом количестве признаков (5, 10). Это связано с тем, что чем меньше признаков выбирается в каждом узле, тем более различными получаются деревья (ведь деревья сильно неустойчивы к изменениям в обучающей выборке), и тем лучше работает их композиция.

5) При большом количестве признаков (40, 50, 60) качество классификации лучше, чем при малом количестве признаков (5, 10). Это связано с тем, что чем больше признаков - тем больше информации об объектах, а значит алгоритм может делать прогнозы более точно.

6) При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса намного лучше, чем без ограничения глубины, т.к. деревья получаются не переобученными. С ростом глубины деревьев качество ухудшается.

7) При небольшой максимальной глубине деревьев (5-6) качество работы случайного леса заметно хуже, чем без ограничений, т.к. деревья получаются недообученными. С ростом глубины качество сначала улучшается, а затем не меняется существенно, т.к. из-за усреднения прогнозов и различий деревьев их переобученность в бэггинге не сказывается на итоговом качестве (все деревья преобучены по-разному, и при усреднении они компенсируют переобученность друг-друга).
How

In [40]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
for estimator in [5,10,15,50,100]:
    bagging_classifier = BaggingClassifier(clf, n_estimators = estimator, bootstrap=False)
    result = cross_val_score(bagging_classifier, X_data, y, cv=cv)
    print 'estimator', estimator, 'result', np.mean(result)

estimator 5 result 0.9505847754012728
estimator 10 result 0.9494053363441857
estimator 15 result 0.9566787040523934
estimator 50 result 0.9533110206067997
estimator 100 result 0.9517174394800227


In [43]:
for max_features in [5, 50]:
    clf = RandomForestClassifier(max_features=max_features)
    bagging_classifier = BaggingClassifier(clf, n_estimators = estimator, bootstrap=False)
    result = cross_val_score(bagging_classifier, X_data, y, cv=cv)
    print 'max_features', max_features, 'result', np.mean(result)

max_features 5 result 0.9577561859685433
max_features 50 result 0.9366155663714177


In [ ]:
for max_depth in [5, 10, 20, 40, 80, 100]:
    clf = RandomForestClassifier(max_depth=max_depth)
    bagging_classifier = BaggingClassifier(clf, n_estimators = estimator, bootstrap=False)
    result = cross_val_score(bagging_classifier, X_data, y, cv=cv)
    print 'max_depth', max_depth, 'result', np.mean(result)

max_depth 5 result 0.9181825068128247
max_depth 10 result 0.9516774882104839
max_depth 20 result 0.954441399394994
max_depth 40 result 0.9544318223468895


In [44]:
write_answer('3 4 7', 5)

write answer #5: 3 4 7
